In [ ]:
import simpy
import random
import statistics

# ---------------- Simulation Setup ----------------
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

# Clinic Details
NUM_COUNTERS = 3
PHARMACISTS_PER_COUNTER = 2
TOTAL_PHARMACISTS = NUM_COUNTERS * PHARMACISTS_PER_COUNTER
SIM_TIME = 480                 # 8 hours (in minutes)

# Peak Day Characteristics
SERVICE_TIME = (6, 8)          # time to serve one patient (minutes)
ARRIVAL_INTERVAL = (0.8, 1.5)  # patients arrive every 0.8–1.5 min (≈500/day)

# ---------------- Simulation Model ----------------
def run_clinic_simulation():
    env = simpy.Environment()
    pharmacists = [simpy.Resource(env, capacity=1) for _ in range(TOTAL_PHARMACISTS)]
    
    wait_times = []
    service_times = []
    queue_samples = []
    served_count = [0] * TOTAL_PHARMACISTS
    busy_time = [0.0] * TOTAL_PHARMACISTS

    # Monitor queue congestion
    def monitor():
        while True:
            total_q = sum(len(p.queue) for p in pharmacists)
            queue_samples.append(total_q)
            yield env.timeout(1)

    # Each patient process
    def patient(env, pid):
        arrival_time = env.now
        pharmacist_id = random.randrange(TOTAL_PHARMACISTS)
        pharmacist = pharmacists[pharmacist_id]

        with pharmacist.request() as req:
            yield req
            wait = env.now - arrival_time
            wait_times.append(wait)

            service_time = random.uniform(*SERVICE_TIME)
            service_times.append(service_time)
            busy_time[pharmacist_id] += service_time
            served_count[pharmacist_id] += 1

            yield env.timeout(service_time)

    # Generate patients over time
    def generate_patients(env):
        pid = 0
        while True:
            pid += 1
            env.process(patient(env, pid))
            yield env.timeout(random.uniform(*ARRIVAL_INTERVAL))
            if env.now > SIM_TIME:
                break

    env.process(monitor())
    env.process(generate_patients(env))
    env.run(until=SIM_TIME)

    # ---------------- Results ----------------
    avg_wait = statistics.mean(wait_times) if wait_times else 0
    avg_service = statistics.mean(service_times) if service_times else 0
    avg_queue = statistics.mean(queue_samples) if queue_samples else 0
    total_served = sum(served_count)
    utilization = [(busy_time[i] / SIM_TIME) * 100 for i in range(TOTAL_PHARMACISTS)]
    avg_utilization = statistics.mean(utilization)

    # ---------------- Output ----------------
    print(" District General Hospital - Horana (Clinic Dispensary)")
    print("=== Peak Day Simulation Results ===")
    print(f"Total Pharmacists: {TOTAL_PHARMACISTS}")
    print(f"Total Patients Served: {total_served}")
    print(f"Average Waiting Time: {avg_wait:.2f} min")
    print(f"Average Processing Time: {avg_service:.2f} min")
    print(f"Average Queue Length: {avg_queue:.2f}")
    print(f"Average Pharmacist Utilization: {avg_utilization:.1f}%")
    print(f"Simulation Duration: {SIM_TIME} minutes")
    print("\n--- Workload per Pharmacist ---")
    for i in range(TOTAL_PHARMACISTS):
        print(f" Pharmacist {i+1}: Served={served_count[i]}, Utilization={utilization[i]:.1f}%")

# ---------------- Run Simulation ----------------
run_clinic_simulation()


🏥 District General Hospital - Horana (Clinic Dispensary)
=== Peak Day Simulation Results ===
Total Pharmacists: 6
Total Patients Served: 384
Average Waiting Time: 19.08 min
Average Processing Time: 6.99 min
Average Queue Length: 17.94
Average Pharmacist Utilization: 93.2%
Simulation Duration: 480 minutes

--- Workload per Pharmacist ---
 Pharmacist 1: Served=61, Utilization=88.9%
 Pharmacist 2: Served=64, Utilization=93.9%
 Pharmacist 3: Served=68, Utilization=98.5%
 Pharmacist 4: Served=64, Utilization=93.6%
 Pharmacist 5: Served=65, Utilization=94.9%
 Pharmacist 6: Served=62, Utilization=89.4%
